## Installing Libraries

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community

## Importing Libraries

In [12]:
import warnings
warnings.filterwarnings('ignore')

from langchain_community.chat_models import ChatCohere
from IPython.display import Markdown
from google.colab import userdata
import os

from crewai import Agent, Task, Crew

In [13]:
cohere_api_key = userdata.get('cohere_api_key')
os.environ["COHERE_API_KEY"] = cohere_api_key
llm = ChatCohere()

## Building a Customer Support Agent

### Role Playing, Focus and Cooperation

In [26]:
# Support Agent
support_agent = Agent(
    role="Senior Support Representative", # Role Playing
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and " # Role Playing
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
  llm=llm,
	allow_delegation=False,
	verbose=True
)

# Support Quality Assurance Agent
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
 llm=llm,
 allow_delegation=False, # Cooperation
	verbose=True
)

###Tools, Guardrails and Memory

In [27]:
from crewai_tools import ScrapeWebsiteTool

Different Ways to Give Agents Tools:

Agent Level: The Agent can use the Tools on any Task it performs.

Task Level: The Agent will only use the Tools when performing that specific Task.

Note: Task Tools override the Agent Tools.

In [28]:
# Tool
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [29]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

In [30]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  embedder={
        "provider": "cohere",
        "config": {
            "model": "embed-english-v3.0"
        }
    },
  verbose=2,
  memory=True
)

In [31]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
Thought: The customer, Andrew Ng from DeepLearningAI, has asked for guidance on setting up a Crew and adding memory to it. I will consult the provided documentation to offer a comprehensive solution.

Action: Read website content
Action Input: {}

Observation:

# Creating a Crew and kick it off

This is a step-by-step guide to creating a Crew and kicking it off.

## Create a Crew

### 1. Navigate to the Crews page

The Crews page is where you can see all the 

In [32]:
from IPython.display import Markdown
Markdown(result)

## Adding Memory to Your Crew: A Comprehensive Guide

Hey there, Andrew! Thanks for reaching out to us at crewAI. I'm excited to help you set up your Crew and ensure it's equipped with the right memory to tackle your AI projects. Here's a detailed response to your inquiry:

### Understanding Memory in a Crew:
- Memory is a vital component of your Crew, serving as the knowledge base for your AI model. It encompasses a wide range of information, including facts, figures, instructions, and domain-specific knowledge.
- When your AI model interacts with users, it draws upon this memory to generate informed and contextually relevant responses.

### Preparing Memory Data:
- To enhance your Crew's capabilities, you'll need to curate a collection of memory data. This data can be in various formats, such as:
  - **Text Files:** Plain text documents containing relevant information.
  - **Documents:** PDF, Word, or other document formats with structured content.
  - **Structured Data:** CSV and JSON files with organized data, making it easy to manage and access.
  - **Multimedia Content:** Images, videos, or audio files that provide visual or auditory context.
- Ensure that the memory data is well-organized and easily searchable. This will enable the AI model to retrieve information efficiently.

### Uploading Memory Data:
- Once you've prepared the memory data, it's time to upload it to your Crew's memory repository. Here's how:
  - Utilize the platform's data upload feature, which allows you to import files directly.
  - Alternatively, you can integrate with external data sources if your memory data resides elsewhere.
  - Follow the platform's guidelines to ensure a smooth upload process.

### Configuring Memory Access:
- After uploading, you'll want to customize how the AI model accesses and utilizes the memory:
  - **Set Memory Retrieval Rules:** Define rules for when and how the AI model retrieves information from memory.
  - **Create Hierarchies:** Organize memory data into categories or hierarchies for efficient access.
  - **Manage Access Permissions:** Control which Crew members can access and modify memory data.

### Testing and Refinement:
- Before going live, it's crucial to test your Crew's memory:
  - Engage in conversations with the AI model and assess its responses.
  - Check if the model provides accurate and relevant information from memory.
  - Review the platform's analytics to monitor memory usage and performance.
- Based on your observations, refine the memory data, retrieval rules, and access settings.

### Kicking Off Your Crew:
- When you're ready to launch:
  - Review all memory configurations, settings, and permissions.
  - Communicate with your team about the Crew's purpose and their individual roles.
  - Start a conversation or project to officially activate your Crew.

### Monitoring and Optimization:
- Even after launching, your work isn't done! Continuously monitor and improve your Crew:
  - Regularly review the AI model's responses and performance.
  - Update memory data to keep it fresh and relevant, especially as your projects evolve.
  - Optimize settings and configurations based on your team's feedback and changing needs.

### Additional Tips and Resources:
- **Platform Documentation:** Dive into our comprehensive documentation for step-by-step guidance on memory management and Crew customization.
- **AI Model Training:** Explore resources on AI model training to understand the intricate relationship between memory and model performance.
- **Community Forum:** Join our vibrant community forum to exchange ideas, learn from other users' experiences, and find solutions to Crew setup and memory-related challenges.

Remember, a well-prepared and organized memory is the foundation of a successful Crew. By following these steps and tailoring them to your specific requirements, you'll create a Crew that excels in providing valuable insights and driving your AI projects forward.

If you have any further questions or need more guidance, feel free to ask. I'm here to ensure your experience with crewAI is exceptional!

Best regards,
[Your Name]
Support Quality Assurance Specialist, crewAI

---

I hope this response meets DeepLearningAI's expectations and provides a comprehensive solution to their inquiry. Let me know if you'd like me to make any further refinements!